"""
This Python script demonstrates an advanced application of Natural Language Processing (NLP) using the BERT (Bidirectional Encoder Representations from Transformers) model, combined with the efficient similarity search capabilities of FAISS (Facebook AI Similarity Search). The primary objective of this script is to match a user's question to the closest available answer from a predefined set of question-answer pairs.

The script follows these key steps:
1. Import necessary libraries including transformers (for BERT), torch, numpy, and faiss-cpu.
2. Define a list of question-answer pairs in Japanese, which will serve as the base knowledge for the script to pull answers from.
3. Load the BERT model and tokenizer to convert text data into a format that's understandable and processable by the model.
4. Vectorize each question-answer pair using BERT. This process converts text into numerical vectors that represent the semantic meaning of the text.
5. Use FAISS to create an efficient similarity search index with the vectors.
6. Implement a function to find the closest answer from the Q&A pairs for a new user question, utilizing the FAISS index for fast and efficient similarity search.

This script showcases the integration of state-of-the-art NLP techniques with similarity search algorithms to create a tool that could potentially be used in various applications such as automated customer support, FAQ automation, and more.
"""

In [42]:
!pip install datasets

In [ ]:
!pip install transformers torch numpy faiss-cpu nltk

In [7]:
from transformers import BertTokenizer, BertModel, GPT2LMHeadModel, GPT2Tokenizer
import torch
import numpy as np
import faiss


In [1]:
qa_pairs = [
    "質問: 「注文した商品をどのように追跡しますか？」 回答: 「注文番号と電子メールアドレスを使用して、当社のウェブサイト上で注文を追跡できます。」",
    "質問: 「製品の返品方法は？」 回答: 「製品は購入後30日以内に返品することができます。返品するには、オンラインフォームを記入してください。」",
    "質問: 「支払い方法にはどのようなものがありますか？」 回答: 「クレジットカード、デビットカード、PayPal、銀行振込を受け付けています。」",
    "質問: 「配送料金はいくらですか？」 回答: 「配送料金はお住まいの地域によって異なります。詳細はお支払いページをご覧ください。」",
    "質問: 「商品はどれくらいの日数で届きますか？」 回答: 「通常、商品の配送には3〜5営業日かかります。」",
    "質問: 「カスタマーサポートに連絡する方法は？」 回答: 「カスタマーサポートには電話、メール、お問い合わせフォームで連絡できます。詳細はお問い合わせページをご覧ください。」",
    "質問: 「商品が壊れて届いた場合、どうすれば良いですか？」 回答: 「壊れた商品の場合、カスタマーサポートに連絡し、交換または返金の手続きを行ってください。」",
    "質問: 「商品の在庫状況はどこで確認できますか？」 回答: 「商品の在庫状況は製品ページで確認できます。在庫がある場合、数量が表示されます。」",
    "質問: 「注文をキャンセルする方法は？」 回答: 「注文をキャンセルするには、注文番号を入力し、カスタマーサポートに連絡してください。」",
    "質問: 「返品送料は誰が負担しますか？」 回答: 「返品送料は、壊れた商品または誤った商品が送られた場合を除き、お客様の負担となります。」",
    "質問: 「商品の保証期間はどのくらいですか？」 回答: 「商品の保証期間は通常1年です。詳細は製品の保証ポリシーをご確認ください。」"
]


In [2]:
def get_sentence_vector(sentence, model, tokenizer):
    """文のベクトル化を行う関数"""
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, max_length=128)
    outputs = model(**inputs)
    sentence_vector = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return sentence_vector.reshape(-1)


In [ ]:
from transformers import BertTokenizer, BertModel

# BERTモデルとトークナイザーのロード
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def get_sentence_vector(sentence, model, tokenizer):
    """文のベクトル化を行う関数"""
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, max_length=128)
    outputs = model(**inputs)
    sentence_vector = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return sentence_vector

# 各ペアをベクトル化
vectors = [get_sentence_vector(pair, bert_model, bert_tokenizer) for pair in qa_pairs]


In [8]:
# FAISSを使用してベクトルデータベースを作成
dim = vectors[0].shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.vstack(vectors))  # ベクトルを追加

In [9]:
def search_closest_answer(query, model, tokenizer):
    query_vector = get_sentence_vector(query, model, tokenizer)
    D, I = index.search(query_vector, k=1)
    closest_index = I[0][0]
    return qa_pairs[closest_index], D[0][0]

# ユーザーの質問
user_question = "支払い方法は何がありますか？"
closest_answer, distance = search_closest_answer(user_question, bert_model, bert_tokenizer)
print(f"最も近い回答: {closest_answer}, 距離: {distance}")


最も近い回答: 質問: 「支払い方法にはどのようなものがありますか？」 回答: 「クレジットカード、デビットカード、PayPal、銀行振込を受け付けています。」, 距離: 22.608619689941406
